In [1]:
! pwd

/Users/skw24/CRUK/clean_landing/CRUK_datahub_landing_page/python_helper_code_and_jsons


In [2]:
import os
import json

In [3]:
path = "../src/utils/longer_filter_data1.js"

In [4]:
with open(path) as f:
    text = f.read().split("\n")

In [8]:
text[1][-50:]

" 'primaryGroup': 'data-type', 'count': '0'}}}}}}}}"

In [9]:
dictionary = eval(text[1])

In [30]:
from typing import Dict, Any, List, Optional

class TreeNode:
    """
    Represents a single node in the hierarchical tree structure.
    """
    def __init__(
        self,
        id: str,
        label: str,
        category: str,
        primary_group: str,
        count: str,
        description: str,
        children: Optional[List['TreeNode']] = None,
    ):
        """
        Initializes a new tree node with data attributes.

        Args:
            id (str): The unique identifier for the node.
            label (str): The human-readable label or name.
            category (str): The category of the node.
            primary_group (str): The primary group classification.
            count (str): A count associated with the node.
            children (Optional[List['TreeNode']]): A list of child TreeNode objects.
        """
        self.id = id
        self.label = label
        self.category = category
        self.primary_group = primary_group
        self.description = description
        self.count = count
        self.children: List['TreeNode'] = children if children is not None else []

    def __repr__(self):
        """
        Provides a string representation for debugging.
        """
        return f"TreeNode(id='{self.id}', label='{self.label}', children={len(self.children)})"

    def print_tree(self, level=0):
        """
        Recursively prints the structure of the tree.
        """
        indent = "  " * level
        print(f"{indent}- {self.label} (ID: {self.id})")
        for child in self.children:
            child.print_tree(level + 1)

In [11]:
def build_dict(tree: TreeNode) -> dict:
    "Recursively builds the new dictionary from the tree"

    dictionary = {}
    for label in ["id", "label", "category", "primary_group", "count"]:
        dictionary[label] = getattr(tree, label)
    if tree.children:
        dictionary["children"] = [build_dict(child) for child in tree.children]
    return dictionary
    

In [12]:
def build_tree(data: Dict[str, Any]) -> TreeNode:
    """
    Recursively builds the TreeNode structure from the raw dictionary data.

    Args:
        data (Dict[str, Any]): The raw dictionary data for a single node.

    Returns:
        TreeNode: The resulting object-oriented tree node.
    """
    # Extract core attributes
    node_id = data.get('id', '')
    label = data.get('label', '')
    category = data.get('category', '')
    primary_group = data.get('primaryGroup', 'cancer-type')
    count = data.get('count', '')
    description = data.get('description', '')

    # Initialize the current node
    current_node = TreeNode(
        id=node_id,
        label=label,
        category=category,
        primary_group=primary_group,
        count=count,
        description=description
    )

    # Check for children and recurse
    raw_children = data.get('children', {})
    if raw_children and isinstance(raw_children, dict):
        # Iterate over the values (the child dictionaries)
        for child_dict in raw_children.values():
            # Recursively build the child node
            child_node = build_tree(child_dict)
            current_node.children.append(child_node)

    return current_node

In [ ]:
tree = build_tree(dictionary['0_2'])

animal = tree.children[2]

animal.label = "Model Organism"

animal.children[0].label = "Organism"
animal.children[1].label = "Organism preparation"

animal.children[0].children[0].primary_group

bakers_yeast = TreeNode(id='0_2_2_0_6', label='Saccharomyces cerevisiae', category='Animal', primary_group='data-type', description="", count=0)
fission_yeast = TreeNode(id='0_2_2_0_7', label='Schizosaccharomyces pombe', category='Animal', primary_group='data-type', description="", count=0)

animal.children[0].children += [bakers_yeast, fission_yeast]

In [34]:
amended_dict = build_dict(tree)
dictionary["0_2"] = amended_dict

In [37]:
path = "../src/utils/longer_filter_data.js"
with open(path, "w") as f:
    f.write(text[0]+"\n")
with open(path,"a") as f:
    json.dump(dictionary, f)
    f.write("\n;\n"+text[2][:-1])

    

In [ ]:
tcga = tree.children.pop()

In [ ]:
for child in tcga.children:
    child.label = f"{child.label} ({child.description})"

In [ ]:
tree.children.append(tcga)

In [ ]:
new_dictionary = build_dict(tree)

In [ ]:
new_dictionary["children"][2]

In [ ]:
new_dictionary["primary_group"]

In [ ]:
dictionary['0_2']

In [ ]:
new_text = text[0]+"\n"+str(dictionary)+"\n"+text[2]+";"

In [ ]:
path = "../src/utils/longer_filter_data1.js"

In [ ]:
with open(path, "w") as f:
    f.write(new_text)

In [ ]:
import json
with open("../src/utils/data.json") as f:
    d = eval(f.read())

In [ ]:
with open("../src/utils/data.json", "w") as f:
    json.dump(d, f)